# Demo phương pháp LSB 

In [1]:
from bitarray import bitarray
from PIL import Image
import numpy as np

In [2]:
def embed(msg_file, cover_img_file, num_lsbs, stego_img_file):
    # Đọc cover img file
    cover_pixels = np.array(Image.open(cover_img_file))
    print(cover_pixels.shape)
    print(cover_pixels.dtype)
    
    # Đọc msg file
    with open(msg_file, 'r') as f:
        msg = f.read()
    print(len(msg))
    
    # Chuyển msg thành msg bits
    msg_bits = bitarray()
    msg_bits.fromstring(msg)
    print(len(msg_bits))
    
    # Kiểm xem có nhúng được không?
    capacity = cover_pixels.size * num_lsbs
    if len(msg_bits) + 1 > capacity:
        print('Cannot embed')
        return False
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (capacity - len(msg_bits) - 1))
   
    # Nhúng msg bits vào cover pixels
    stego_pixels = np.empty_like(cover_pixels)
    for r in range(cover_pixels.shape[0]):
        for c in range(cover_pixels.shape[1]):
            b = (r * cover_pixels.shape[1] + c) * num_lsbs
            stego_pixels[r, c] = (cover_pixels[r, c]>>num_lsbs<<num_lsbs)\
                                + int(msg_bits[b:b+num_lsbs].to01(), 2)
    
    # Ghi stego pixels xuống file
    Image.fromarray(stego_pixels).save(stego_img_file)
    
    return True
    
embed('05-msg.txt', '05-cover.bmp', 2, '05-stego.bmp')

(512, 512)
uint8
727
5816


PermissionError: [Errno 13] Permission denied: '05-stego.bmp'

In [11]:
def extract(stego_img_file, num_lsbs, extr_msg_file):
    # Đọc stego img file
    stego_pixels = np.array(Image.open(stego_img_file))
    
    # Rút trích msg bits
    extr_msg_bits = bitarray()
   
    for pixel in stego_pixels.reshape(-1):
        extr_msg_bits.extend((np.binary_repr(pixel & (2**num_lsbs-1), num_lsbs)))
    pixel = 10101100, num_ = 2

    # Cắt đuôi '100...' ra khỏi msg bits
    extr_msg_bits = extr_msg_bits[:extr_msg_bits.to01().rfind('1')]
    
    # Chuyển msg bits thành msg
    extr_msg = extr_msg_bits.tostring()
    
    # Ghi msg xuống file
    with open(extr_msg_file, 'w') as f:
        f.write(extr_msg)
        
extract('05-stego.bmp', 2, '05-extr_msg.txt')